Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
from pathlib import Path
# Add the directory containing your script to the Python path
sys.path.append(r'C:\Users\User\Project-SLMonlineControl\PYsubfun')
#sys.path.append(r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS

In [2]:
#importlib.reload(FBS)
# Pre-set parameters, and load the only bin files in the pre-set path folde
ConfigFolder=r'C:\Users\User\Project-SLMonlineControl\config'
#ConfigFolder=r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\config'
ops0, confSet=FBS.configLoad(ConfigFolder,'SLMsetting.yml','ops3Planes.npy')
SaveFolder=FBS.suite2pInitiate(ops0,ConfigFolder)
binFile = glob.glob(ops0['save_path0'] + '/TSeries*-001.bin')    ##<<<<<<<<<<<<<<<<<<<<<<<<<< Noted that there supposed be only one Bin Data file
#Load Data
print(binFile)
rawBin, FramePerPlane, TotalFrameNeed=FBS.LoadBin(binFile,ops0)
rawBin.shape

Processed data would be saved inE:\LuSLMOnlineTest\SL0840-Ai203\02242025\suite2p
File found: C:\Users\User\Project-SLMonlineControl\config\SLMsetting.yml
Copying to: E:\LuSLMOnlineTest\SL0840-Ai203\02242025\CurrentSLMsetting.yml
Current SLMsetting copied successfully!
Folder already exists.
['E:\\LuSLMOnlineTest\\SL0840-Ai203\\02242025\\TSeries-02242025-0910-001.bin']


(24600, 512, 512)

In [6]:
ops0['spatial_scale']=2
ops0['threshold_scaling']=1.1
ops0['denoise']=1

In [5]:
confSet

{'save_path0': 'E:\\LuSLMOnlineTest\\SL0840-Ai203\\02242025\\',
 'SLM_Pixels_X': 512,
 'SLM_Pixels_Y': 512,
 'scan_Z': 171.74,
 'ETL': '-50 0 50',
 'IsSpiral': 'True',
 'SpiralSizeUM': 15,
 'SpiralRevolution': 3,
 'Repetition': 5,
 'Duration': 20,
 'InterPointDelay': 30,
 'UncagingLaserPower': '1.45 1.55 1.65',
 'InitialDelay': 22.1,
 'NumNonTarget': 20,
 'NumNTperTrial': 9,
 'NumTrial': 5,
 'RadiusAvoidParam': 3,
 'nchannels': 1,
 'fs': 6.9,
 'tau': 0.9,
 'Ziteration': 11,
 'ZRepetition': 34}

Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [7]:
# Define the number of parallel processes; 
# Noted that current binning movie in chuns of length 06; increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes=1              ## <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Create a list of plane indices
plane_indices = range(0,ops0['nplanes'])
print(plane_indices)
ops1=ops0
Parallel(n_jobs=num_processes)(delayed(FBS.process_plane)(plane_idx, rawBin, SaveFolder, ops1) for plane_idx in plane_indices)

# Combine multiple plane data
#FBS.CombinePlanes(SaveFolder,ops0)
FBS.CombinePlanes(SaveFolder,ops0)

range(0, 3)
Processing plane 0
Binning movie in chunks of length 06
Binned movie of size [1366,512,512] created in 9.43 sec.
Binned movie denoised (for cell detection only) in 38.19 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.14, threshold 12.52 
0 ROIs, score=84.51
Detected 68 ROIs, 23.13 sec
After removing overlaps, 68 ROIs remain
added enhanced mean image
Masks created, 0.56 sec.


c:\users\user\suite2p\suite2p\extraction\extract.py:125: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  Fi[n] = np.dot(data[:, cell_ipix[n]], cell_lam[n])


Extracted fluorescence from 68 ROIs in 8200 frames, 15.57 sec.
Extracted fluorescence from 68 ROIs in 8200 frames, 10.16 sec.
['compact', 'skew', 'npix_norm']
Processing plane 1
Binning movie in chunks of length 06
Binned movie of size [1366,512,512] created in 8.32 sec.
Binned movie denoised (for cell detection only) in 25.53 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.14, threshold 12.52 
0 ROIs, score=77.15
Detected 68 ROIs, 23.55 sec
After removing overlaps, 68 ROIs remain
added enhanced mean image
Masks created, 0.53 sec.
Extracted fluorescence from 68 ROIs in 8200 frames, 10.18 sec.
Extracted fluorescence from 68 ROIs in 8200 frames, 10.20 sec.
['compact', 'skew', 'npix_norm']
Processing plane 2
Binning movie in chunks of length 06
Binned movie of size [1366,512,512] created in 8.19 sec.
Binned movie denoised (for cell detection only) in 26.24 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.14, threshold 12.52 
0 ROIs, score=73.54
Detected 79 ROIs, 24.26 s

Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [8]:
FBS.PostMannual(SaveFolder,ops0)

215 of units including 112 identified cells in combined planes
